# Generating new biz model points for savings

This notebook is modified from *generate_model_points.ipynb* and
generates the sample model points for the `BasicTerm_SE` and `BasicTerm_ME` model, by using random numbers.
The model ponints have the `duration_mth` attribute, which indicates how many months elapsed from the
issue of each model point to time 0. Negative `duration_mth` indicate future new business.


**Columns:**

* `point_id`: Model point identifier
* `age_at_entry`: Issue age. The samples are distributed uniformly from 20 to 59.
* `sex`: "M" or "F" to indicate policy holder's sex. Not used by default.
* `policy_term`: Policy term in years. The samples are evenly distriubted among 10, 15 and 20.
* `policy_count`: The number of policies. Uniformly distributed from 0 to 100.
* `sum_assured`: Sum assured. The samples are uniformly distributed from 10,000 to 1,000,000.
* `duration_mth`: Months elapsed from the issue til t=0. Negative values indicate future new business. Uniformly distributed from -36 to 12 times `policy_term`.  

**Number of model points:**

* 100,000


In [7]:
import numpy as np
from numpy.random import default_rng  # Requires NumPy 1.17 or newer

rng = default_rng(12345)

# Number of Model Points
MPCount = 100000

# Issue Age (Integer): 20 - 59 year old
age_at_entry = rng.integers(low=20, high=60, size=MPCount)

# Sex (Char)
Sex = [
    "M",
    "F"
]

sex = np.fromiter(map(lambda i: Sex[i], rng.integers(low=0, high=len(Sex), size=MPCount)), np.dtype('<U1'))

# Policy Term (Integer): 10, 15, 20 
policy_term = rng.integers(low=0, high=3, size=MPCount) * 5 + 10

 
# Sum Assured (Float): 10000 - 1000000  
sum_assured = np.round((1000000 - 10000) * rng.random(size=MPCount) + 10000, -3)

# Duration in month (Int): -36 < Duration(mth) < Policy Term in month
duration_mth = np.rint((policy_term + 3) * 12 * rng.random(size=MPCount) - 36).astype(int)

# Policy Count (Integer): 1
policy_count = np.rint(100 * rng.random(size=MPCount)).astype(int)

In [2]:
import pandas as pd

attrs = [
    "age_at_entry",
    "sex",
    "policy_term",
    "policy_count",
    "sum_assured",
    "duration_mth"
]

data = [
    age_at_entry,
    sex,
    policy_term,
    policy_count,
    sum_assured,
    duration_mth
]

model_point_table = pd.DataFrame(dict(zip(attrs, data)), index=range(1, MPCount+1))
model_point_table.index.name = "policy_id"
model_point_table

,age_at_entry,sex,policy_term,policy_count,sum_assured,duration_mth
policy_id,,,,,,
1,47,M,20,62,804000.0,26
2,29,F,20,26,519000.0,171
3,51,F,10,44,409000.0,-28
4,32,M,15,85,128000.0,94
5,28,M,20,16,698000.0,89
...,...,...,...,...,...,...
99996,21,M,10,21,152000.0,17
99997,24,F,15,77,928000.0,78
99998,46,F,15,67,662000.0,120


In [3]:
# Spec ID
SpedIDs = "ABCD"

spec_ids = np.fromiter(map(lambda i: SpedIDs[i], rng.integers(low=0, high=len(SpedIDs), size=MPCount)), np.dtype('<U1'))
model_point_table.insert(0, 'spec_id', spec_ids)

whole_life = (model_point_table['spec_id'] == 'C')  | (model_point_table['spec_id'] == 'D')

# Modify Policy Term

model_point_table['policy_term'].mask(whole_life, 9999, inplace=True)

# Set Duration to 0

model_point_table['duration_mth'] = 0

# Premium per Plicy

premium_pp = np.ceil(model_point_table['sum_assured'] / (80 - model_point_table['age_at_entry']) / 12 / 100) * 100

model_point_table.insert(len(model_point_table.columns), 'premium_pp', model_point_table['sum_assured'])
model_point_table['premium_pp'].mask(whole_life, premium_pp, inplace=True)

# Ininital Account Value per Policy 

model_point_table['av_pp_init'] = 0


In [4]:
model_point_table

,spec_id,age_at_entry,sex,policy_term,policy_count,sum_assured,duration_mth,premium_pp,av_pp_init
policy_id,,,,,,,,,
1,B,47,M,20,62,804000.0,0,804000.0,0
2,C,29,F,9999,26,519000.0,0,900.0,0
3,B,51,F,10,44,409000.0,0,409000.0,0
4,A,32,M,15,85,128000.0,0,128000.0,0
5,B,28,M,20,16,698000.0,0,698000.0,0
...,...,...,...,...,...,...,...,...,...
99996,C,21,M,9999,21,152000.0,0,300.0,0
99997,A,24,F,15,77,928000.0,0,928000.0,0
99998,A,46,F,15,67,662000.0,0,662000.0,0


In [8]:
model_point_table.to_excel('model_point_table_100K.xlsx')